In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import joblib
import datetime
calendar_df = pd.read_csv('calendar.csv')
sales_df = pd.read_csv('sales_train_validation.csv')
prices_df = pd.read_csv('sell_prices.csv')


In [17]:
from sklearn.linear_model import SGDRegressor
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

In [4]:
#dataframe = pd.read_pickle('reduced.pickle')

dataframe = pd.read_pickle('preprocessed.pickle')


In [5]:
#trying to train with just last 13 days (1901-1913)
#dataframe = dataframe[dataframe['d']>1912]
dataframe = dataframe[dataframe['d']>1100]
dataframe = dataframe[dataframe['d']<1885]

In [6]:
dataframe.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23904160 entries, 33539000 to 57443159
Data columns (total 41 columns):
 #   Column             Dtype         
---  ------             -----         
 0   id                 category      
 1   item_id            category      
 2   dept_id            category      
 3   cat_id             category      
 4   store_id           category      
 5   state_id           category      
 6   d                  int16         
 7   sales              int16         
 8   date               datetime64[ns]
 9   wm_yr_wk           int16         
 10  weekday            category      
 11  wday               int8          
 12  month              int8          
 13  year               int8          
 14  event_name_1       category      
 15  event_type_1       category      
 16  event_name_2       category      
 17  event_type_2       category      
 18  snap_CA            category      
 19  snap_TX            category      
 20  snap_WI        

In [6]:
dataframe.shape

(23904160, 41)

In [7]:
#label encoding to get rid of string values from data
le = LabelEncoder()
for column in dataframe.columns:
    dtype = dataframe[column].dtypes
    if dtype == 'category':
        print("Transforming", column)
        le.fit(dataframe[column])
        dataframe[column] = le.transform(dataframe[column])
dataframe.head()

Transforming id
Transforming item_id
Transforming dept_id
Transforming cat_id
Transforming store_id
Transforming state_id
Transforming weekday
Transforming event_name_1
Transforming event_type_1
Transforming event_name_2
Transforming event_type_2
Transforming snap_CA
Transforming snap_TX
Transforming snap_WI


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,rolling_mean_7,rolling_median_7,rolling_mean_14,rolling_median_14,rolling_mean_28,rolling_median_28,rolling_mean_35,rolling_median_35,rolling_mean_60,rolling_median_60
33539000,14370,14370,14370,14370,14370,14370,1101,0,2014-02-02,11401,...,0.285645,0.0,0.214233,0.0,0.606934,0.0,0.600098,0.0,0.649902,0.0
33539001,14380,14380,14380,14380,14380,14380,1101,0,2014-02-02,11401,...,0.000000,0.0,0.000000,0.0,0.357178,0.0,0.285645,0.0,0.199951,0.0
33539002,14390,14390,14390,14390,14390,14390,1101,0,2014-02-02,11401,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
33539003,14400,14400,14400,14400,14400,14400,1101,2,2014-02-02,11401,...,2.285156,2.0,2.572266,2.0,2.464844,2.0,2.427734,2.0,2.333984,2.0
33539004,14410,14410,14410,14410,14410,14410,1101,1,2014-02-02,11401,...,0.428467,0.0,0.785645,0.0,0.893066,0.0,0.885742,1.0,0.833496,0.5


In [8]:
#splitting train data and train label
labels = dataframe['sales']
train = dataframe.drop(labels='sales', axis=1)

In [9]:
#dropping date for now cuz error
train = train.drop(labels='date', axis=1)

In [11]:
print(datetime.datetime.now())
rf = RandomForestRegressor(n_jobs=-2, verbose=True ,n_estimators=50,max_depth=10)
rf.fit(train, labels)
print(datetime.datetime.now())

2022-10-29 13:12:49.647192


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=-2)]: Done  20 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-2)]: Done  50 out of  50 | elapsed: 43.0min finished


2022-10-29 13:56:01.947434


In [12]:
joblib.dump(rf, '13daysmode2_785_10.pkl')

['13daysmode2_785_10.pkl']

In [10]:

#WRMSSE
def convert_to_42840(df, cols, groupbys):
    series_gen = {}
    for i, grp in enumerate(groupbys):
        #grop by corresponding group and calculating aggregate sales of each day
        tmp = df.groupby(grp)[cols].sum()
        #storing the aggregate sale values of each corresponding group
        for j in range(len(tmp)):
            series_gen[gen_series_name(tmp.index[j])] = tmp.iloc[j].values
    return pd.DataFrame(series_gen).T #creating a dataframe of each corresponding group and aggregate sales each day i.e., transformed into 42840 sales

#this method return the name of each group
def gen_series_name(name):
    if isinstance(name, str) | isinstance(name, int):
        return str(name)
    else:
        return "__".join(name)

#here we are computing weights using the last 28 day sales of train data and their prices
def compute_weights(train_df,valid_df,weight_cols,groupbys,fix_cols):
    weights_map = {}
    weight_df = train_df[["item_id", "store_id"] + weight_cols]
    weight_df = pd.melt(weight_df,id_vars=["item_id", "store_id"],var_name='d',value_name='sales')
    weight_df = weight_df.merge(calendar_df[['wm_yr_wk','d']], on='d', how='left')
    weight_df = weight_df.merge(prices_df, how="left", on=["item_id", "store_id", "wm_yr_wk"])
    #computing dollar sales 
    weight_df["dollar_sales"] = weight_df["sales"] * weight_df["sell_price"]
    weight_df = weight_df.set_index(["item_id", "store_id", "d"]).unstack(level=2)["dollar_sales"]
    weight_df = weight_df.loc[zip(train_df.item_id, train_df.store_id), :].reset_index(drop=True)
    weight_df = pd.concat([train_df[fix_cols], weight_df],
                          axis=1, sort=False)
    #computing the weights for each group keys
    for i,grp in enumerate(groupbys):
        ser_weight = weight_df.groupby(grp)[weight_cols].sum().sum(axis=1)
        ser_weight = ser_weight / ser_weight.sum()
        for j in range(len(ser_weight)):
            weights_map[gen_series_name(ser_weight.index[j])] = np.array([ser_weight.iloc[j]])
    weights = pd.DataFrame(weights_map).T / len(groupbys) #creating a dataframe with weights corresponding to each group keys of 42840 hierachical time-series
    return weights

#here we compute the rmsse using the true values and predicted values along with train data which is being used to scale
#train data is used to scale the squared-error as taking the consecutive difference of each day sales
def compute_rmsse(train_df, valid_df, pred_df):
    scale_lst = []
    for i in range(len(train_df)):
        val = train_df.iloc[i].values
        # to consider the periods following the first non-zero demand observed for the series under evaluation.
        val = val[np.argmax(val!=0):]
        #to scale the squared-error as taking the consecutive difference of each day sales
        scale = ((val[1:] - val[:-1]) ** 2).mean()
        #storing the scale value corresponding to each time series
        scale_lst.append(scale)
    scale_arr = np.array(scale_lst)
    #computing mean squared error
    num = ((pred_df - valid_df)**2).mean(axis=1)
    #scaled error i.e., root mean squared scaled error
    rmsse = (num/scale_arr).map(np.sqrt)
    return rmsse

#here we return the final score value i.e., WRMSSE
def custom_metric(train_df, valid_df, pred_df, weights):
    #obtaing RMSSE by calling compute_rmsse function
    rmsse = compute_rmsse(train_df, valid_df, pred_df)
    #WRMSSE of each 42840 time-series is computed as product of corresponding weights and RMSSE respectively
    ser_metric = pd.concat([weights, rmsse], axis=1, sort=False).prod(axis=1)
    return np.sum(ser_metric) #aggregation of each WRMSSE of 42840 time-series to get the final WRMSSE score

In [49]:
test = pd.read_pickle('preprocessed.pickle')

le = LabelEncoder()
for column in test.columns:
    dtype = test[column].dtypes
    if dtype == 'category':
        print("Transforming", column)
        le.fit(test[column])
        test[column] = le.transform(test[column])
labels1 = test['sales']        
test = test.drop(labels='date', axis=1)
test = test.drop(labels='sales', axis=1)
test = test[test['d']>1885]


test.head()


Transforming id
Transforming item_id
Transforming dept_id
Transforming cat_id
Transforming store_id
Transforming state_id
Transforming weekday
Transforming event_name_1
Transforming event_type_1
Transforming event_name_2
Transforming event_type_2
Transforming snap_CA
Transforming snap_TX
Transforming snap_WI


,id,item_id,dept_id,cat_id,store_id,state_id,d,wm_yr_wk,weekday,wday,...,rolling_mean_7,rolling_median_7,rolling_mean_14,rolling_median_14,rolling_mean_28,rolling_median_28,rolling_mean_35,rolling_median_35,rolling_mean_60,rolling_median_60
57473650,14370,14370,14370,14370,14370,14370,1886,11609,1,3,...,1.142578,1.0,1.142578,1.0,1.000000,1.0,0.942871,1.0,0.883301,0.5
57473651,14380,14380,14380,14380,14380,14380,1886,11609,1,3,...,0.142822,0.0,0.214233,0.0,0.142822,0.0,0.171387,0.0,0.150024,0.0
57473652,14390,14390,14390,14390,14390,14390,1886,11609,1,3,...,0.000000,0.0,0.214233,0.0,0.321533,0.0,0.285645,0.0,0.250000,0.0
57473653,14400,14400,14400,14400,14400,14400,1886,11609,1,3,...,1.857422,1.0,1.857422,1.5,2.107422,2.0,2.228516,2.0,2.066406,2.0
57473654,14410,14410,14410,14410,14410,14410,1886,11609,1,3,...,1.000000,1.0,0.928711,1.0,1.178711,1.0,1.114258,1.0,0.916504,0.5


In [35]:

rf2 = joblib.load("13daysmodel.pkl")
#rf = pd.read_pickle('13daysmode2_785_10.pkl')
preds1 = pd.DataFrame()
#predict for day 1914
for i in range(1886,1914):
    preds1['d_'+str(i)] = rf2.predict(test[test['d']==i])
#df['d_'+str(i)] =rf2.predict(test)
#df=rf.predict(train[train['d']==1801])
preds1.head(10)

[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    1.1s
[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:    1.4s finished
[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    1.3s
[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:    2.2s finished
[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.1s
[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1.38,1.24,0.70,0.48,1.04,1.22,1.06,1.42,0.98,0.88,...,1.20,0.88,1.22,1.60,1.12,1.08,1.16,1.32,1.10,1.52
1,0.54,0.48,0.42,0.42,0.36,0.40,0.56,0.78,0.68,0.70,...,0.54,0.54,0.26,0.46,0.54,0.50,0.50,0.70,0.54,0.46
2,0.52,0.70,1.22,1.14,0.42,1.18,0.82,1.26,1.36,0.80,...,0.58,0.76,0.48,0.98,0.84,0.48,0.56,0.74,0.82,1.48
3,1.88,1.62,1.72,1.42,2.72,2.18,2.28,1.44,1.78,1.20,...,2.18,2.04,2.18,2.12,1.76,2.14,1.52,2.26,3.94,3.74
4,1.14,0.92,0.94,1.18,0.80,1.34,1.50,0.90,0.88,1.28,...,1.22,1.76,1.80,1.10,1.40,0.98,1.50,0.96,0.96,1.48
5,1.44,1.58,2.64,0.78,1.36,1.88,1.02,2.30,1.24,2.08,...,2.04,1.32,1.10,0.74,1.84,1.96,1.78,1.64,1.48,1.78
6,0.76,0.74,0.60,0.62,0.52,0.72,0.82,0.28,0.40,0.42,...,0.86,0.62,0.38,0.78,0.52,0.50,0.42,0.58,0.52,0.58
7,11.92,12.94,11.62,17.18,11.12,15.26,15.14,12.80,9.76,8.84,...,9.34,12.94,10.94,7.60,4.86,11.82,5.60,5.80,9.58,2.56
8,0.02,0.00,1.20,0.54,0.76,0.26,0.32,0.60,0.54,0.68,...,0.56,0.84,0.80,1.14,0.34,0.50,0.40,0.16,0.98,0.88
9,0.76,0.88,0.64,0.66,0.92,0.50,0.90,0.70,0.66,0.68,...,0.98,0.62,0.74,0.86,0.60,0.60,0.56,0.82,0.90,1.44


,id,item_id,dept_id,cat_id,store_id,state_id,d_1069,d_1070,d_1071,d_1072,...,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,for_all
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,1,0,0,...,1,3,1,2,2,0,1,1,1,all
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,1,1,1,all
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,1,1,0,all
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,2,4,4,2,...,2,1,4,1,3,5,0,6,6,all
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,...,2,2,2,3,1,0,0,0,0,all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,1,2,2,...,0,0,0,0,0,0,0,0,0,all
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,all
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,2,0,0,0,2,0,1,all
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,3,3,0,...,1,1,1,0,2,1,1,2,4,all


In [38]:

#preds = pd.concat([valid_df[fixed_cols], preds],axis=1,sort=False)
#group-bys to group into 12-level hierarchical time series to calculate WRMSSE 
groupbys = ('for_all', 'state_id', 'store_id', 'cat_id', 'dept_id',['state_id', 'cat_id'],  
            ['state_id', 'dept_id'], ['store_id', 'cat_id'],['store_id', 'dept_id'], 'item_id', 
            ['item_id', 'state_id'], ['item_id', 'store_id'])
train_df = pd.concat([sales_df.loc[:,:'state_id'],sales_df.loc[:,'d_1069':]],axis=1,sort=False)
print(train_df.head(100))
valid_df = train_df.iloc[:,-28:].copy()
train_df = train_df.iloc[:,:-28]
print(train_df.head(100))
print(valid_df.head(100))
train_d_cols = [col for col in train_df.columns if col.startswith('d_')]
weight_cols = train_df.iloc[:,-28:].columns.tolist()
train_df['for_all'] = "all" #for level 1 aggregation
fixed_cols = [col for col in train_df.columns if not col.startswith('d_')]
valid_d_cols = [col for col in valid_df.columns if col.startswith('d_')]
if not all([col in valid_df.columns for col in fixed_cols]):
    valid_df = pd.concat([train_df[fixed_cols],valid_df],axis=1,sort=False)
#weights corresponding all the 12-level hierarchical transformed 42840 time-series
weight_df = compute_weights(train_df,valid_df,weight_cols,groupbys,fixed_cols)
#train data transformed from 30490 timeseries to 42840 hirerachichal time-series
train_42840_df = convert_to_42840(train_df, train_d_cols, groupbys)
#validation data transformed from 30490 timeseries to 42840 hirerachichal time-series
valid_42840_df = convert_to_42840(valid_df, valid_d_cols, groupbys)


                               id        item_id    dept_id   cat_id store_id  \
0   HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1   HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2   HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3   HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4   HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   
..                            ...            ...        ...      ...      ...   
95  HOBBIES_1_099_CA_1_validation  HOBBIES_1_099  HOBBIES_1  HOBBIES     CA_1   
96  HOBBIES_1_100_CA_1_validation  HOBBIES_1_100  HOBBIES_1  HOBBIES     CA_1   
97  HOBBIES_1_102_CA_1_validation  HOBBIES_1_102  HOBBIES_1  HOBBIES     CA_1   
98  HOBBIES_1_103_CA_1_validation  HOBBIES_1_103  HOBBIES_1  HOBBIES     CA_1   
99  HOBBIES_1_104_CA_1_validation  HOBBIES_1_104  HOBBIES_1  HOBBIES     CA_1   

   state_id  d_1069  d_1070

In [39]:
valid_df.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,for_all,d_1886,d_1887,d_1888,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,all,1,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,all,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,all,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,all,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,all,1,0,4,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,all,0,1,3,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,all,1,2,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,all,0,11,5,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,all,0,2,0,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,all,0,1,1,...,1,0,0,0,0,0,0,2,0,2


In [40]:
preds1.head(10)

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1.38,1.24,0.70,0.48,1.04,1.22,1.06,1.42,0.98,0.88,...,1.20,0.88,1.22,1.60,1.12,1.08,1.16,1.32,1.10,1.52
1,0.54,0.48,0.42,0.42,0.36,0.40,0.56,0.78,0.68,0.70,...,0.54,0.54,0.26,0.46,0.54,0.50,0.50,0.70,0.54,0.46
2,0.52,0.70,1.22,1.14,0.42,1.18,0.82,1.26,1.36,0.80,...,0.58,0.76,0.48,0.98,0.84,0.48,0.56,0.74,0.82,1.48
3,1.88,1.62,1.72,1.42,2.72,2.18,2.28,1.44,1.78,1.20,...,2.18,2.04,2.18,2.12,1.76,2.14,1.52,2.26,3.94,3.74
4,1.14,0.92,0.94,1.18,0.80,1.34,1.50,0.90,0.88,1.28,...,1.22,1.76,1.80,1.10,1.40,0.98,1.50,0.96,0.96,1.48
5,1.44,1.58,2.64,0.78,1.36,1.88,1.02,2.30,1.24,2.08,...,2.04,1.32,1.10,0.74,1.84,1.96,1.78,1.64,1.48,1.78
6,0.76,0.74,0.60,0.62,0.52,0.72,0.82,0.28,0.40,0.42,...,0.86,0.62,0.38,0.78,0.52,0.50,0.42,0.58,0.52,0.58
7,11.92,12.94,11.62,17.18,11.12,15.26,15.14,12.80,9.76,8.84,...,9.34,12.94,10.94,7.60,4.86,11.82,5.60,5.80,9.58,2.56
8,0.02,0.00,1.20,0.54,0.76,0.26,0.32,0.60,0.54,0.68,...,0.56,0.84,0.80,1.14,0.34,0.50,0.40,0.16,0.98,0.88
9,0.76,0.88,0.64,0.66,0.92,0.50,0.90,0.70,0.66,0.68,...,0.98,0.62,0.74,0.86,0.60,0.60,0.56,0.82,0.90,1.44


In [41]:
preds1 = pd.concat([valid_df[fixed_cols], preds1],axis=1,sort=False)
valid_d_cols

['d_1886',
 'd_1887',
 'd_1888',
 'd_1889',
 'd_1890',
 'd_1891',
 'd_1892',
 'd_1893',
 'd_1894',
 'd_1895',
 'd_1896',
 'd_1897',
 'd_1898',
 'd_1899',
 'd_1900',
 'd_1901',
 'd_1902',
 'd_1903',
 'd_1904',
 'd_1905',
 'd_1906',
 'd_1907',
 'd_1908',
 'd_1909',
 'd_1910',
 'd_1911',
 'd_1912',
 'd_1913']

In [42]:
WRMSSE_rf = {}
pred_42840_df = convert_to_42840(preds1,valid_d_cols,groupbys)
    #Computed WRMSSE for each predictions based on hyper-parameters
WRMSSE_rf[(50,10)] = custom_metric(train_42840_df,valid_42840_df,pred_42840_df,weight_df)


In [43]:
print('WRMSSE for n_estimator 50 with depth=10 is:\n',WRMSSE_rf)

WRMSSE for n_estimator 50 with depth=10 is:
 {(50, 10): 1.2811768026401942}
